In [4]:
# Import necessary libraries
import ee
import geemap
import datetime
from ipyleaflet import Map, WidgetControl
from ipywidgets import Output

# Function to initialize Google Earth Engine
def initialize_ee(project_id):
    try:
        ee.Initialize(project=project_id)
    except ee.EEException:
        ee.Authenticate()
        ee.Initialize(project=project_id)

In [7]:


# Function to compute date ranges per year
def compute_date_ranges(start_date, end_date):
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")

    # Generate the list of date ranges
    date_ranges = []
    current_year = start.year

    while current_year <= end.year:
        range_start = start_date if current_year == start.year else f"{current_year}-01-01"
        range_end = end_date if current_year == end.year else f"{current_year}-12-31"
        date_ranges.append([range_start, range_end])
        current_year += 1

    return date_ranges

# Function to define the MODIS data processing pipeline
def process_modis_data(start_date, end_date, southern_california):
    # Parse reference date
    reference_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    reference_date_millis = int(reference_date.timestamp() * 1000)

    # Compute date ranges
    date_ranges = compute_date_ranges(start_date, end_date)
    #print(date_ranges)

    # Function to create a GeoTIFF for the given year and extract pixel data
    def create_tiff_and_extract_data(range_start, range_end):
        # Load the MODIS data for the specified range
        modis_burned_area = ee.ImageCollection('MODIS/061/MCD64A1')\
                              .filter(ee.Filter.date(range_start, range_end))\
                              .filterBounds(southern_california)

        burned_area = modis_burned_area.select('BurnDate')
        latest_burn = burned_area.reduce(ee.Reducer.max()).rename('latest_burn_date')
        clipped_burn = latest_burn.clip(southern_california)

        # Function to convert Julian day to actual date
        def julian_to_date(image):
            burn_date = image.select('latest_burn_date')
            start_of_year_millis = ee.Date(range_start).millis()
            return burn_date.expression(
                'burn_date == 0 ? 0 : startOfYear + (burn_date - 1) * 86400000', {
                    'burn_date': burn_date,
                    'startOfYear': start_of_year_millis
                }).rename("yearly_burn_date")

        return julian_to_date(clipped_burn).toFloat()

    # Aggregate burn data for all date ranges
    most_recent_burn = ee.Image(0).rename('most_recent_burn_date').clip(southern_california).toFloat()

    for range_start, range_end in date_ranges:
        yearly_burn_date = create_tiff_and_extract_data(range_start, range_end)
        most_recent_burn = most_recent_burn.where(
            yearly_burn_date.gt(most_recent_burn), yearly_burn_date
        )

    # Calculate days since the last burn date
    days_since_burn = most_recent_burn.expression(
        'burn_date == 0 ? 0 : round((reference_date - burn_date) / (1000 * 60 * 60 * 24))', {
            'reference_date': reference_date_millis,
            'burn_date': most_recent_burn
        }
    ).rename("days_since_last_burn").toFloat()

    combined_image = most_recent_burn.addBands(days_since_burn)
    return combined_image

def calculate_days(start_date, end_date):
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    return (end - start).days
    
# Function to visualize the data on a map
def visualize_map(start_date, end_date, combined_image, palette, southern_california, center):
    max_days = calculate_days(start_date, end_date)
    # Define visualization parameters
    vis_params = {
        'bands': 'days_since_last_burn',
        'min': 1,
        'max': max_days,
        'palette': palette
    }

    # Create a map centered on Southern California
    Map = geemap.Map(center=center, zoom=10)

    # Mask out areas with no data
    combined_image = combined_image.updateMask(combined_image.select('days_since_last_burn').neq(0))

    # Add the combined image with visualization parameters
    Map.addLayer(combined_image, vis_params, "Days Since Last Burn")

    # Add interactivity
    output = Output()
    output_control = WidgetControl(widget=output, position='topright')
    Map.add_control(output_control)

    def handle_click(**kwargs):
      with output:
        output.clear_output()  # Clear any previous output
        event_type = kwargs.get('type')  # Check event type

        # Process only if it's a 'click' event
        if event_type != 'click':
            return

        # Extract latitude and longitude from 'coordinates' in kwargs
        coordinates = kwargs.get('coordinates')
        if coordinates is None:
            print("No location data available.")
            return

        lat, lon = coordinates[0], coordinates[1]
        print(f"Latitude: {lat}, Longitude: {lon}")  # Display coordinates

        # Create a point geometry
        point = ee.Geometry.Point([lon, lat])

        # Sample the combined image at this location
        values = combined_image.sample(
            region=point,
            scale=20,
            geometries=True
        ).first()

        try:
            info = values.getInfo()
            burn_date = info['properties'].get('most_recent_burn_date')
            days_since_last_burn = info['properties'].get('days_since_last_burn')

            # Print the retrieved data
            print(f"Most Recent Burn Date: {burn_date}")
            print(f"Days Since Last Burn: {days_since_last_burn}")
        except Exception as e:
            print("No data available for this point.", str(e))
    # Attach click event to the map
    Map.on_interaction(handle_click)
    return Map

In [8]:
# Project id from Google Earth Engine
initialize_ee("test-09112024")

# Define variables
start_date = "2000-01-01"
end_date = "2024-09-30"
southern_california = ee.Geometry.BBox(-120.708008, 33.156797, -113.975687, 38.395343)
center=[35.5688, -118.8678]
palette = ['red', 'orange', 'yellow', 'green', 'blue', 'purple'] #recent color to oldest color

# Process data and visualize
combined_image = process_modis_data(start_date, end_date, southern_california)
Map = visualize_map(start_date, end_date, combined_image, palette, southern_california, center)
Map

Map(center=[35.5688, -118.8678], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…